Before start

We use Gemini 1.5 Flash (simmilar to Claude3 Haiku, but compare its Cost effectiveness : Gemini 1.5 Flash is much better in 2024. may 16)

`pip install -q -U google-generativeai`

In [1]:
## setup gemini with gcp sdk

import google.generativeai as genai
print([model.name for model in genai.list_models() if 'generateContent' in model.supported_generation_methods])

['models/gemini-1.0-pro', 'models/gemini-1.0-pro-001', 'models/gemini-1.0-pro-latest', 'models/gemini-1.0-pro-vision-latest', 'models/gemini-1.5-flash-latest', 'models/gemini-1.5-pro-latest', 'models/gemini-pro', 'models/gemini-pro-vision']


In [2]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest")

In [3]:
response = model.generate_content("Hello world!") #check working
print(response.text)

Hello there! 👋  How can I help you today? 😊 



Working well. now use it to owr usecase.

In [4]:
## Before start, load parsed document from my parsing method: table handling with tatr
from pathlib import  Path
from document_preprocess import process_pdf_text_from_plumber, postprocess_with_datr, remove_duplicates

base_path = Path("preprocess_document/output")
pdf_path = base_path / "sample.pdf"

detection_path = base_path / "results/detection"
structure_path = base_path / "results/structure"

# base_path의 마지막 디렉토리 이름과 pdf_path의 파일 이름(확장자를 제외한 부분)을 결합하여 file_prefix 생성
file_prefix = f"{base_path.stem}_{pdf_path.stem}"

pdfplumber_extracted_text = process_pdf_text_from_plumber(pdf_path, detection_path, file_prefix)

# for element in pdfplumber_extracted_text:
#     print(f"Type: {element.type}, Page: {element.page_number}, Table Index: {element.table_index}")
#     print(element.text)
#     print("-" * 80)

postprocess_result = postprocess_with_datr(pdfplumber_extracted_text, structure_path, file_prefix)

print(postprocess_result) ## returned List[TextElement].

[TextElement(type='text', text='Supply specification KWN 49037 Revision: A Material and testing requirements Forgings ', page_number=0, table_index=None, is_table=False), TextElement(type='table', text='| Created by: | Dipl.-Ing. M. Krondorf | 03/06/2022 | sgd. M. Krondorf |\n| :--- | :--- | :--- | :--- |\n| Examined by: | Dr.-Ing. T. Hähnel | 03/06/2022 | sgd. T. Hähnel |\n|  | Name | Date | Signature |\n', page_number=0, table_index=0, is_table=True), TextElement(type='text', text='KWD Kupplungswerk Dresden GmbH Löbtauer Straße 45 D – 01159 Dresden Postfach 270144 D – 01172 Dresden Tel.: + 49(0)351 – 4999-0 Fax.: + 49(0)351 – 4999-233 kwd@kupplungswerk-dresden.de http://www.kupplungswerk-dresden.de KWN49037_A_eng.docx Page 1 von 13 Copyright (\uf0e3) KWD Kupplungswerk Dresden, All rights reserved. Supply specification ', page_number=0, table_index=None, is_table=False), TextElement(type='table', text='| Supply specification KWN 49037 |\n| :--- |\n|\n', page_number=1, table_index=0, i

In [5]:
## Now we setup section and whole context.
## It will be summarize section based on whole context(cause model's input context is large enough)

## simple whole context setup
whole_context = "\n\n".join(element.text for element in postprocess_result)

section_list = [element for element in postprocess_result if element.type=="table"]
unique_section = remove_duplicates(section_list)

Checking element: | Created by: | Dipl.-Ing. M. Krondorf | 03/06/2022 | sgd. M. Krondorf |
| :--- | :--- | :--- | :--- |
| Examined by: | Dr.-Ing. T. Hähnel | 03/06/2022 | sgd. T. Hähnel |
|  | Name | Date | Signature |

Checking element: | Supply specification KWN 49037 |
| :--- |
|

Checking element: | Supply specification KWN 49037 |
| :--- |
| Revision: A |

Checking element: | Group | Inclusion thickness | Max. permitted class |
| :--- | :--- | :--- |
| A | Fine | 3 |
| Thick | 3 |
| B | Fine | 2,5 |
| Thick | 1,5 |
| C | Fine | 2,5 |
| Thick | 1,5 |
| D | Fine | 2 |
| Thick | 1,5 |
| DS |  | 2 |

Checking element: | Class | Inclusion group |
| :--- | :--- |
|
| 0,5 | 37 | 17 | 18 | 1 | 13 |
| 1 | 127 | 77 | 76 | 4 | 19 |
| 1,5 | 261 | 184 | 176 | 9 | 27 |
| 2 | 436 | 343 | 320 | 16 | 38 |
| 2,5 | 649 | 555 | 510 | 25 | 53 |
| 3 | 898 | 822 | 746 | 36 | 76 |

Checking element: | Inclusion group | Fine | Thick |
| :--- | :--- | :--- |
|
| A | 2 | 4 | 4 | 12 |
| B | 2 | 9 | 9 | 15 |

In [13]:
# print(element.text for element in section_list)

<generator object <genexpr> at 0x0000020051E68F90>


In [16]:
print("\n\n".join([element.text for element in section_list]))

| Created by: | Dipl.-Ing. M. Krondorf | 03/06/2022 | sgd. M. Krondorf |
| :--- | :--- | :--- | :--- |
| Examined by: | Dr.-Ing. T. Hähnel | 03/06/2022 | sgd. T. Hähnel |
|  | Name | Date | Signature |


| Supply specification KWN 49037 |
| :--- |
|


| Supply specification KWN 49037 |
| :--- |
| Revision: A |


| Group | Inclusion thickness | Max. permitted class |
| :--- | :--- | :--- |
| A | Fine | 3 |
| Thick | 3 |
| B | Fine | 2,5 |
| Thick | 1,5 |
| C | Fine | 2,5 |
| Thick | 1,5 |
| D | Fine | 2 |
| Thick | 1,5 |
| DS |  | 2 |


| Class | Inclusion group |
| :--- | :--- |
|
| 0,5 | 37 | 17 | 18 | 1 | 13 |
| 1 | 127 | 77 | 76 | 4 | 19 |
| 1,5 | 261 | 184 | 176 | 9 | 27 |
| 2 | 436 | 343 | 320 | 16 | 38 |
| 2,5 | 649 | 555 | 510 | 25 | 53 |
| 3 | 898 | 822 | 746 | 36 | 76 |


| Inclusion group | Fine | Thick |
| :--- | :--- | :--- |
|
| A | 2 | 4 | 4 | 12 |
| B | 2 | 9 | 9 | 15 |
| C | 2 | 5 | 5 | 12 |
| D | Ø2 | Ø 8 | Ø 8 | Ø 13 |


| Standards | Application area |
| :--- | :--- 

In [8]:
print(len(unique_section))



16


In [16]:
print(section_list)

[TextElement(type='table', text='| Created by: | Dipl.-Ing. M. Krondorf | 03/06/2022 | sgd. M. Krondorf |\n| :--- | :--- | :--- | :--- |\n| Examined by: | Dr.-Ing. T. Hähnel | 03/06/2022 | sgd. T. Hähnel |\n|  | Name | Date | Signature |\n', page_number=0, table_index=0, is_table=True), TextElement(type='table', text='| Supply specification KWN 49037 |\n| :--- |\n|\n', page_number=1, table_index=0, is_table=True), TextElement(type='table', text='| Supply specification KWN 49037 |\n| :--- |\n| Revision: A |\n', page_number=2, table_index=0, is_table=True), TextElement(type='table', text='| Group | Inclusion thickness | Max. permitted class |\n| :--- | :--- | :--- |\n| A | Fine | 3 |\n| Thick | 3 |\n| B | Fine | 2,5 |\n| Thick | 1,5 |\n| C | Fine | 2,5 |\n| Thick | 1,5 |\n| D | Fine | 2 |\n| Thick | 1,5 |\n| DS |  | 2 |\n', page_number=3, table_index=0, is_table=True), TextElement(type='table', text='| Class | Inclusion group |\n| :--- | :--- |\n|\n| 0,5 | 37 | 17 | 18 | 1 | 13 |\n| 1 | 

In document summarization, unlike video summarization like on YouTube, the process was carried out by viewing the entire document and sectioning parts of it, without the need for predefined sections. In this case, the parts being sectioned are tables.

In [ ]:
BASE_PROMPT = f""" 
You are Summarizer. Based on given whole document and section, you only summarize section based on whole document.
You can provide more info about table by compare-and summarize

############################################################################# WHOLE DOCUMENT
{whole_context}
#############################################################################

you only summarize section based on whole document.

############################################################################# SECTION
{section}
#############################################################################
"""

In [3]:
model = genai.GenerativeModel(model_name="models/gemini-pro-vision")

In [5]:
from PIL import Image

# 이미지 파일을 열고 표시합니다.
img = Image.open('preprocess_document/output/results/detection/output_sample_page2_table_0.png')

In [15]:
gemini_result = model.generate_content(["tell me all the detail about this table.", img], stream=True)

In [19]:
gemini_result.resolve()
gemini_result.text

' The table shows the heat treatment and physical properties of a metal. The heat treatment process involves hardening the metal by quenching it in water at a temperature of 820-870 degrees Celsius, followed by tempering it by forced cooling at a temperature of 550-650 degrees Celsius. The physical properties of the metal after heat treatment are as follows: yield strength of 50 kg/mm2, tensile strength of 70 kg/mm2, elongation of 17%, reduction of area of 45%, Charpy impact test of 8 kgfm/cm2, and hardness Brinell of 201-269 HB.'